# Regression model in Keras
## A - Build a baseline model

### Import libraries

In [1]:
import pandas as pd
import numpy as np

### Load data

In [2]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Check dataset

In [3]:
concrete_data.shape

(1030, 9)

In [4]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [5]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Sanity check

In [6]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [7]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Number of independent variables

In [8]:
n_cols = predictors.shape[1]

Randomly split the data into a training and test sets by holding 30% of the data for testing

### Additional step for B: normalize data

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)
print('Training set shapes: {} {}'.format(X_train.shape, y_train.shape))
print('Testing set shapes: {} {}'.format(X_test.shape, y_test.shape))

Training set shapes: (721, 8) (721,)
Testing set shapes: (309, 8) (309,)


### Build a neural network

Characteristics:

* One hidden layer of 10 nodes, and a ReLU activation function
* Use the **adam** optimizer and the **mean squared error**  as the loss function

In [11]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

Define regression model (Case D: 3 hidden layers)

In [12]:
def baseline_regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Compile the model

In [13]:
model = baseline_regression_model()

### Train the model on the normalized training data using 50 epochs

In [14]:
model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=1)

Train on 504 samples, validate on 217 samples
Epoch 1/50
504/504 [==============================] - 1s 2ms/step - loss: 1609.1899 - val_loss: 1476.6919
Epoch 2/50
504/504 [==============================] - 0s 706us/step - loss: 1593.2775 - val_loss: 1458.9716
Epoch 3/50
504/504 [==============================] - 0s 503us/step - loss: 1573.3873 - val_loss: 1436.9788
Epoch 4/50
504/504 [==============================] - 0s 448us/step - loss: 1549.2096 - val_loss: 1407.8237
Epoch 5/50
504/504 [==============================] - 0s 438us/step - loss: 1516.0656 - val_loss: 1369.4890
Epoch 6/50
504/504 [==============================] - 0s 523us/step - loss: 1472.4669 - val_loss: 1318.8607
Epoch 7/50
504/504 [==============================] - 0s 517us/step - loss: 1415.2460 - val_loss: 1255.5677
Epoch 8/50
504/504 [==============================] - 0s 514us/step - loss: 1342.0195 - val_loss: 1176.8950
Epoch 9/50
504/504 [==============================] - 0s 474us/step - loss: 1251.4344 - val_

### Evaluate the model on the normalized test data
Compute the mean squared error between the predicted concrete strength and the actual concrete strength

In [15]:
from sklearn.metrics import mean_squared_error
yhat = model.predict(X_test)
print("MSE: %.4f" % mean_squared_error(y_test , yhat))

MSE: 119.3276


### Repeat splitting, training and evaluation 50 times

Create a list of 50 MSE (don't forget to normalize)

In [16]:
mse_list = list()

for i in range(50):
    # Train test split
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)
    # Compile and train the model
    model = baseline_regression_model()
    model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=0)
    # Predict based on test set and calculate MSE
    yhat = model.predict(X_test)
    mse = mean_squared_error(y_test , yhat)
    mse_list.append(mse)
    print("Iteration #%d --- MSE: %.4f" % (i+1, mse))

Iteration #1 --- MSE: 140.9516
Iteration #2 --- MSE: 141.1927
Iteration #3 --- MSE: 149.4404
Iteration #4 --- MSE: 138.3745
Iteration #5 --- MSE: 152.5499
Iteration #6 --- MSE: 141.6761
Iteration #7 --- MSE: 132.9654
Iteration #8 --- MSE: 139.6274
Iteration #9 --- MSE: 148.3543
Iteration #10 --- MSE: 141.9607
Iteration #11 --- MSE: 142.6252
Iteration #12 --- MSE: 140.7928
Iteration #13 --- MSE: 157.9390
Iteration #14 --- MSE: 152.2507
Iteration #15 --- MSE: 155.0378
Iteration #16 --- MSE: 134.7968
Iteration #17 --- MSE: 147.2025
Iteration #18 --- MSE: 138.1682
Iteration #19 --- MSE: 137.1715
Iteration #20 --- MSE: 156.9727
Iteration #21 --- MSE: 152.2563
Iteration #22 --- MSE: 143.6638
Iteration #23 --- MSE: 138.9471
Iteration #24 --- MSE: 154.6431
Iteration #25 --- MSE: 150.7455
Iteration #26 --- MSE: 143.1762
Iteration #27 --- MSE: 133.1164
Iteration #28 --- MSE: 133.7625
Iteration #29 --- MSE: 142.0956
Iteration #30 --- MSE: 151.6139
Iteration #31 --- MSE: 150.0335
Iteration #32 ---

### Report the mean and the standard deviation of the mean squared errors

In [17]:
print("Mean of MSE: %.4f" % np.mean(mse_list))
print("Standard deviation of MSE: %.4f" % np.std(mse_list))

Mean of MSE: 143.4663
Standard deviation of MSE: 8.0976


The mean of MSE in case D (143.4663) was ~4.5 times less than the mean of MSE for normalized data in case B (652.7311)